In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


155.9386761188507


In [4]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0015211105346679688


In [5]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.5076537132263184


In [6]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

3.973339080810547


In [7]:
#remove columns which have any null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

45.6051390171051


In [8]:
len(unclean_columns)

103

In [9]:
unclean_columns

Index(['installation_date', 'product_sales_order_type',
       'product_purchase_order_type', 'service_contract_last_renewal_date',
       'service_contract_classification', 'service_contract_process_type',
       'contract_line_duration_in_days', 'marketing_program_name',
       'service_contract_discount_percentage', 'warranty_type',
       ...
       'service_distributor_base_distributor_type',
       'service_distributor_base_country_name',
       'service_distributor_base_theater_name',
       'service_distributor_base_distributor_normalized_name',
       'sales_node_renewal_rate', 'customer_renewal_rate',
       'partner_renewal_rate', 'product_renewal_rate',
       'service_sales_node_installed_base_sales_node_renewal_rate',
       'service_partner_installed_base_partner_renewal_rate'],
      dtype='object', length=103)

In [10]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.649034023284912


In [11]:
# 56 messy columns are removed
len(messy_columns)

53

In [12]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.0462229251861572


In [13]:
train.shape

(226140, 80)

In [14]:
#Remove the primary key column for data fitting
t1=time.time()
train_X = train.drop('innovation_challenge_key',axis=1)
test_X = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.0836155414581299


In [15]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train_X.columns:
    if (train_X[col].dtype !='object'):
        if(len(train_X[col].unique())<20):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.012831449508666992


In [16]:
#handle catcols
t1=time.time()
for col in catcols:
    train_X[col]=train_X[col].astype('category')
    test_X[col]=test_X[col].astype('category')
t2=time.time()
print(t2-t1)

0.15447425842285156


In [17]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train_X[scalcols])
train_X[scalcols] = scaler.transform(train_X[scalcols])
scaler = scaler.fit(test_X[scalcols])
test_X[scalcols] = scaler.transform(test_X[scalcols])
t2=time.time()
print(t2-t1)

0.8672370910644531


In [18]:
train_X[scalcols].describe()

,contract_line_net_usd_amount,product_net_price
count,2.261400e+05,2.261400e+05
mean,1.005455e-17,1.206546e-17
std,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01
25%,-1.252316e-01,-2.321716e-01
50%,-1.252316e-01,-2.321716e-01
75%,-1.252316e-01,-1.584198e-01
max,7.774224e+01,6.648806e+01


In [19]:
train_X[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [20]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_X)
dataset = pd.concat(objs=[train_X, test_X], axis=0)
dataset = pd.get_dummies(dataset)
train_X = dataset[:train_objs_num]
test_X = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

15.280014514923096


In [21]:
train_X.columns

Index(['contract_line_reaction_time_code',
       'service_sales_node_base_sales_hierarchy_level',
       'contract_line_net_usd_amount', 'product_net_price',
       'minor_line_yorn_N', 'minor_line_yorn_Y',
       'instance_status_Latest-INSTALLED', 'product_transaction_type_-9999',
       'product_transaction_type_10000', 'product_transaction_type_10002',
       ...
       'service_distributor_base_distributor_name_56BDC7DCC4764EF615A779B492373FA5E9675DD93AC63A03883B9D611D9C1E03',
       'service_distributor_base_distributor_name_63E656F07CCA14C7E92A9CF75C45C2F0E5BDFD3A07A1EF9EC3E92F5AF051FA22',
       'service_distributor_base_distributor_name_7F3282462F1B81C79FE6937D9B49AF13384D961DDF42DF6CF1BB59D7630308A2',
       'service_distributor_base_distributor_name_815A7B66FE919A14C089996194F98F696B39D83332E31755073B836FEA52C364',
       'service_distributor_base_distributor_name_84DCB9C6A3A305DBE0088EDCF4C4BC00C13208407ADA44DFBECEA93D98E8D4B3',
       'service_distributor_base_distributor

In [22]:
#print the new columns we got
train_X.shape

(226140, 370)

In [23]:
test_X.shape

(1888195, 370)

In [24]:
#Replace all NaN left with 0
t1=time.time()
train_X = train_X.fillna(0)
test_X = test_X.fillna(0)
t2=time.time()
print(t2-t1)

0.28705453872680664


In [25]:
#check data quality now
train_X.isnull().any().any()

False

In [26]:
#check data quality now
test_X.isnull().any().any()

False

In [27]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_X,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.4989480972290039


In [28]:
#Dimensions of training data
X_train.shape

(151513, 370)

In [29]:
#Lets start fitting different models on the data
#1. start with simple Bayes model
t1=time.time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gaussian_model = gnb.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

1.4430644512176514


In [30]:
# Lets see the prediction on our validation data
t1=time.time()
pred = gaussian_model.predict_proba(X_valid)
t2=time.time()
print(t2-t1)

0.5990414619445801


In [31]:
pred[1,]

array([1.15020444e-09, 9.99999999e-01])

In [32]:
train_y.head()

0    N
1    N
2    N
3    N
4    N
Name: renewed_yorn, dtype: object

In [33]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

8.414693836815434

In [34]:
#find predicted value on test set
pred = gaussian_model.predict_proba(test_X)

In [39]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]

In [40]:
os.chdir('/users/hdpsndbx125')

In [41]:
#very important to sort the values else won't be taken at submission
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')

In [42]:
#persist to a file
submit.to_csv('NaiveBayesGaussian_EntireTraining.csv',index=False)

In [43]:
submit.shape

(1888195, 2)

In [46]:
#fitting a multinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
#find predicted value on test set
pred = clf.predict_proba(test_X)

In [47]:
from sklearn.naive_bayes import BernoulliNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train,y_train)
pred = mnb.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [35]:
from sklearn.linear_model import LogisticRegression as lr
t1=time.time()
lr1=lr(max_iter=100)
lr1.fit(X_train,y_train)
pred = lr1.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5700277541457386
34.96129059791565


In [36]:
#find predicted value on test set
pred = lr1.predict_proba(test_X)

In [38]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('LR_100-iter-all.csv',index=False)

In [2]:
submit.shape

NameError: name 'submit' is not defined

In [39]:
from sklearn.linear_model import SGDClassifier
t1=time.time()
sgd = SGDClassifier(loss="log", penalty="l2",max_iter=1000)
sgd.fit(X_train,y_train)
pred = sgd.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5705484853625451
230.78573322296143


In [44]:
print(log_loss(y_valid,pred[:,1]))

0.5705468084851013


In [46]:
#trying Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
t1=time.time()
rfc = RandomForestClassifier(max_depth=4, random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5818079529053721
1.519083023071289


In [29]:
pred = rfc.predict_proba(test_X)

In [30]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('RFC_depth4.csv',index=False)

In [ ]:
from sklearn import svm
import time
t1=time.time()
svc= svm.SVC(kernel='rbf',probability=True,verbose=True)
svc.fit(X_train, y_train)
t2=time.time()
pred = svc.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

[LibSVM]

In [1]:
log_loss(y_valid,pred[:,1])

NameError: name 'log_loss' is not defined

In [49]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier()
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(train_X, train_y)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()

In [50]:
t2-t1

148.6567051410675

In [51]:
pred=xgb1.predict_proba(X_valid)

In [52]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

0.5479801843282507

In [53]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test_X)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-entiretrainscl-allcol.csv',index=False)

In [57]:
os.listdir('.')

['XGB-entiretrainscl-allcol.csv',
 'LR_100iter.csv',
 'scoring_set.tsv',
 'LR_100-iter-all.csv',
 'training_set.tsv',
 'challengeData.tsv',
 'lost+found']

In [30]:
#Trying MLPerceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [33]:
t1=time.time()
mlp.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

MemoryError: 

In [43]:
from sklearn.metrics import log_loss
pred = mlp.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.4238643172502673

In [33]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']

In [35]:
pred=mlp.predict_proba(test_X)

In [37]:
len(pred)

1888195

In [38]:
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('MLP_20_20_20_scl.csv',index=False)